# 1. 데이터 & 라이브러리 불러오기

In [1]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
)
import optuna

import os
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
train = pd.read_csv("dataset/train.csv").drop(columns=["ID"])
test = pd.read_csv("dataset/test.csv").drop(columns=["ID"])

In [3]:
train_data = train.copy()
test_data = test.copy()

# 2. 데이터 전처리 - 1

### 특정 시술 유형 재범주화

In [4]:
train_data[['특정 시술 유형']].dtypes

특정 시술 유형    object
dtype: object

In [5]:
def categorize_procedure(procedure):
    if isinstance(procedure, str):
        if "ICSI" in procedure:
            return "ICSI"
        elif "IVF" in procedure:
            return "IVF"
        elif "IUI" in procedure:
            return "IUI"
        else:
            return "Other"
    else:
        return "Unknown"

train_data["시술_카테고리"] = train_data["특정 시술 유형"].apply(categorize_procedure)
test_data["시술_카테고리"] = test_data["특정 시술 유형"].apply(categorize_procedure)

In [6]:
train_data[['시술_카테고리']].value_counts()

시술_카테고리
ICSI       128547
IVF         94568
Other       27134
IUI          6100
Unknown         2
Name: count, dtype: int64

### 시술 복합 여부 변수

In [7]:
train_data["시술_복합여부"] = train_data["특정 시술 유형"].fillna("Unknown").apply(lambda x: 1 if "/" in x else 0)
test_data["시술_복합여부"] = test_data["특정 시술 유형"].fillna("Unknown").apply(lambda x: 1 if "/" in x else 0)

In [8]:
train_data[['시술_복합여부']].value_counts()

시술_복합여부
0          252391
1            3960
Name: count, dtype: int64

### 시술 성공률 변수

In [9]:
# 시술 유형별 성공률 계산
success_rates = train_data.groupby("특정 시술 유형")["임신 성공 여부"].mean()

# 기존 데이터에 성공률 변수 추가
train_data["시술_성공률"] = train_data["특정 시술 유형"].map(success_rates)
test_data["시술_성공률"] = test_data["특정 시술 유형"].map(success_rates)

In [10]:
train_data = train_data.drop(columns = "특정 시술 유형")
test_data = test_data.drop(columns = "특정 시술 유형")

### 주/부 불임 원인 줄이기

In [11]:
train_data["남성_불임_유무"] = train_data[["남성 주 불임 원인", "남성 부 불임 원인"]].max(axis=1)
train_data["여성_불임_유무"] = train_data[["여성 주 불임 원인", "여성 부 불임 원인"]].max(axis=1)
train_data["부부_불임_유무"] = train_data[["부부 주 불임 원인", "부부 부 불임 원인"]].max(axis=1)
train_data["불임_조합"] = (
    train_data["남성_불임_유무"].astype(str) + "_" +
    train_data["여성_불임_유무"].astype(str)
)

test_data["남성_불임_유무"] = test_data[["남성 주 불임 원인", "남성 부 불임 원인"]].max(axis=1)
test_data["여성_불임_유무"] = test_data[["여성 주 불임 원인", "여성 부 불임 원인"]].max(axis=1)
test_data["부부_불임_유무"] = test_data[["부부 주 불임 원인", "부부 부 불임 원인"]].max(axis=1)
test_data["불임_조합"] = (
    test_data["남성_불임_유무"].astype(str) + "_" +
    test_data["여성_불임_유무"].astype(str)
)

In [12]:
주부불임원인 = [col for col in train_data.columns if "주 불임 원인" in col or "부 불임 원인" in col]
train_data = train_data.drop(columns=주부불임원인)
test_data = test_data.drop(columns=주부불임원인)

### 불임원인

In [13]:
train_data["배란장애_여부"] = train_data["불임 원인 - 배란 장애"]
train_data["정자_문제_여부"] = train_data[["불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]].max(axis=1)
train_data["자궁_문제_여부"] = train_data[["불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증"]].max(axis=1)
train_data["난관_문제_여부"] = train_data["불임 원인 - 난관 질환"]

test_data["배란장애_여부"] = test_data["불임 원인 - 배란 장애"]
test_data["정자_문제_여부"] = test_data[["불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]].max(axis=1)
test_data["자궁_문제_여부"] = test_data[["불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증"]].max(axis=1)
test_data["난관_문제_여부"] = test_data["불임 원인 - 난관 질환"]

In [14]:
불임원인질환 = [col for col in train_data.columns if "불임 원인 -" in col]
train_data = train_data.drop(columns = 불임원인질환)
test_data = test_data.drop(columns = 불임원인질환)

### 배란 유도 유형

In [15]:
def 배란유도유형(procedure):
    if isinstance(procedure, str):
        if "생식선 자극 호르몬" in procedure:
            return "생식선 자극 호르몬"
        elif "세트로타이드 (억제제)" in procedure:
            return "세트로타이드 (억제제)"
        else:
            return "Unknown"
    else:
        return "Unknown"

train_data["배란유도유형"] = train_data["배란 유도 유형"].apply(배란유도유형)
test_data["배란유도유형"] = test_data["배란 유도 유형"].apply(배란유도유형)

In [16]:
train_data = train_data.drop(columns = "배란 유도 유형")
test_data = test_data.drop(columns = "배란 유도 유형")

### 배아 생성 주요 이유 -> 원핫인코딩화

In [17]:
def 배아생성주요이유(df):
    # NaN 값을 "Unknown"으로 대체
    df["배아 생성 주요 이유"] = df["배아 생성 주요 이유"].fillna("Unknown")

    # 1️⃣ 배아 생성 이유를 리스트로 변환
    df["배아 생성 이유 리스트"] = df["배아 생성 주요 이유"].apply(lambda x: x.split(", ") if x != "Unknown" else ["Unknown"])

    # 2️⃣ 고유한 배아 생성 이유 추출
    unique_reasons = set()
    df["배아 생성 이유 리스트"].apply(lambda x: unique_reasons.update(x))

    # 3️⃣ 각 배아 생성 이유에 대한 원-핫 인코딩 변수 생성
    for reason in unique_reasons:
        df[f"배아_이유_{reason}"] = df["배아 생성 이유 리스트"].apply(lambda x: 1 if reason in x else 0)

    # 불필요한 컬럼 제거
    if "배아_이유_연구용" in df.columns:
        df.drop(columns=["배아 생성 이유 리스트", "배아 생성 주요 이유", "배아_이유_연구용"], inplace=True)
    else:
        df.drop(columns=["배아 생성 이유 리스트", "배아 생성 주요 이유"], inplace=True)
    return df

In [18]:
train_data = 배아생성주요이유(train_data)
test_data = 배아생성주요이유(test_data)

### 배아 저장 비율

In [19]:
train_data["배아_저장_비율"] = train_data["저장된 배아 수"] / train_data["총 생성 배아 수"]
test_data["배아_저장_비율"] = test_data["저장된 배아 수"] / test_data["총 생성 배아 수"]

### 연속형으로

In [20]:
col_to_convert = ['IVF 시술 횟수', 'IVF 임신 횟수', 'IVF 출산 횟수', 'DI 시술 횟수', 'DI 임신 횟수', 'DI 출산 횟수', '총 임신 횟수', '총 출산 횟수', '총 시술 횟수']

for col in col_to_convert:
    train_data[col] = train_data[col].replace("6회 이상", "7")
    train_data[col] = train_data[col].astype(str).str[0].astype(int)

In [45]:
for col in col_to_convert:
    test_data[col] = test_data[col].replace("6회 이상", "7")
    test_data[col] = test_data[col].astype(str).str[0].astype(int)

## 컬럼 지정

In [ ]:
Top 10 Feature Importances:
Feature: 이식된 배아 수, Importance: 36.2623, Unique Values: 4
Feature: 배아 이식 경과일, Importance: 19.9380, Unique Values: 9
Feature: 시술 당시 나이, Importance: 6.8832, Unique Values: 7
Feature: 난자 채취 경과일, Importance: 5.1645, Unique Values: 1
Feature: 배아_저장_비율, Importance: 2.8436, Unique Values: 305
Feature: 총 생성 배아 수, Importance: 2.8407, Unique Values: 44
Feature: 저장된 배아 수, Importance: 2.2780, Unique Values: 41
Feature: 수집된 신선 난자 수, Importance: 1.8074, Unique Values: 52
Feature: 시술 시기 코드, Importance: 1.5020, Unique Values: 7
Feature: 난자 출처, Importance: 1.4110, Unique Values: 3

In [56]:
numerical_cols = [
 #'임신 시도 또는 마지막 임신 경과 연수',
 '총 생성 배아 수',
 '미세주입된 난자 수',
 '미세주입에서 생성된 배아 수',
 '이식된 배아 수',
 '미세주입 배아 이식 수',
 '저장된 배아 수',
 '미세주입 후 저장된 배아 수',
 '해동된 배아 수',
 '해동 난자 수',
 '수집된 신선 난자 수',
 '저장된 신선 난자 수',
 '혼합된 난자 수',
 '파트너 정자와 혼합된 난자 수',
 '기증자 정자와 혼합된 난자 수',
 '난자 채취 경과일',
 '난자 해동 경과일',
 '난자 혼합 경과일',
 #'배아 이식 경과일',   # 범주형으로 빼는 게 나을 것 같아서..
 '배아 해동 경과일',
 '시술_성공률',  # 추가해본 변수
 '배아_저장_비율' # 추가해본 변수
 ]
for col in col_to_convert:
    numerical_cols.append(col)
categorical_cols = list(
    set(train_data.columns) - set(numerical_cols) - set(["임신 성공 여부"])
)

In [57]:
numerical_cols

['총 생성 배아 수',
 '미세주입된 난자 수',
 '미세주입에서 생성된 배아 수',
 '이식된 배아 수',
 '미세주입 배아 이식 수',
 '저장된 배아 수',
 '미세주입 후 저장된 배아 수',
 '해동된 배아 수',
 '해동 난자 수',
 '수집된 신선 난자 수',
 '저장된 신선 난자 수',
 '혼합된 난자 수',
 '파트너 정자와 혼합된 난자 수',
 '기증자 정자와 혼합된 난자 수',
 '난자 채취 경과일',
 '난자 해동 경과일',
 '난자 혼합 경과일',
 '배아 해동 경과일',
 '시술_성공률',
 '배아_저장_비율',
 'IVF 시술 횟수',
 'IVF 임신 횟수',
 'IVF 출산 횟수',
 'DI 시술 횟수',
 'DI 임신 횟수',
 'DI 출산 횟수',
 '총 임신 횟수',
 '총 출산 횟수',
 '총 시술 횟수']

In [58]:
# train, test 모두 categorical은 str로 변경
train_data[categorical_cols] = train_data[categorical_cols].astype("str")
test_data[categorical_cols] = test_data[categorical_cols].astype("str")

X = train_data.drop("임신 성공 여부", axis=1)
y = train_data["임신 성공 여부"]

# 모델링

In [60]:
def CAT(params, seed):

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=seed
    )

    # 모델 초기화 및 학습(모델 시드는 77로 고정)
    model = CatBoostClassifier(
        **params,
        random_state=77,
        verbose=0,
        cat_features=categorical_cols,
        early_stopping_rounds=150,
    )
    # 최종 검증 데이터셋에서 모델 성능 평가
    model.fit(X_train, y_train)
    y_pred_probs = model.predict_proba(X_val)[:, 1]
    y_pred_final = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred_final)
    precision = precision_score(y_val, y_pred_final)
    recall = recall_score(y_val, y_pred_final)
    f1_final = f1_score(y_val, y_pred_final)
    conf_matrix = confusion_matrix(y_val, y_pred_final)
    roc_auc_final = roc_auc_score(y_val, y_pred_probs)

    # 결과 출력
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_final:.4f}")
    print(f"ROC-AUC Score: {roc_auc_final:.4f}")  # 🎯 ROC-AUC 출력
    print("\nConfusion Matrix:")
    print(conf_matrix)

    # 변수 중요도 계산
    feature_importances = model.get_feature_importance()

    # 변수 이름과 중요도 및 nunique 값을 zip으로 묶은 후 중요도 기준으로 내림차순 정렬
    sorted_importances = sorted(
        zip(X_train.columns, feature_importances, X_train.nunique()),
        key=lambda x: x[1],
        reverse=True,
    )

    # 상위 10개 변수 중요도 및 nunique 값 출력
    print("\nTop 10 Feature Importances:")
    for feature_name, importance, nunique in sorted_importances[:10]:
        print(
            f"Feature: {feature_name}, Importance: {importance:.4f}, Unique Values: {nunique}"
        )

    return roc_auc_final

In [63]:
def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 4, 9),
        "n_estimators": trial.suggest_int("n_estimators", 500, 1800),
        "class_weights": [1, trial.suggest_float("class_weights", 1.5, 2, step=0.1)],
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 3, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.5)
    }

    # 시드 설정
    seed = 2468

    roc_auc_score = CAT(params, seed)

    return roc_auc_score


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best parameters found: ", study.best_params)

[I 2025-02-27 01:31:10,423] A new study created in memory with name: no-name-ce9fd0cf-6267-4345-926f-10fc1cb52c54


Accuracy: 0.7105
Precision: 0.4474
Recall: 0.5131
F1 Score: 0.4780
ROC-AUC Score: 0.7415

Confusion Matrix:
[[44443 12594]
 [ 9674 10195]]


[I 2025-02-27 01:41:25,205] Trial 0 finished with value: 0.7414968661878563 and parameters: {'learning_rate': 0.05628905543120771, 'depth': 6, 'n_estimators': 1205, 'class_weights': 1.8, 'l2_leaf_reg': 9, 'bagging_temperature': 1.3970969442483785}. Best is trial 0 with value: 0.7414968661878563.



Top 10 Feature Importances:
Feature: 이식된 배아 수, Importance: 37.8525, Unique Values: 4
Feature: 배아 이식 경과일, Importance: 23.8305, Unique Values: 9
Feature: 시술 당시 나이, Importance: 6.4387, Unique Values: 7
Feature: 난자 채취 경과일, Importance: 5.3236, Unique Values: 1
Feature: 총 생성 배아 수, Importance: 2.8095, Unique Values: 44
Feature: 배아_저장_비율, Importance: 2.5184, Unique Values: 305
Feature: 수집된 신선 난자 수, Importance: 2.1232, Unique Values: 52
Feature: 난자 출처, Importance: 1.7738, Unique Values: 3
Feature: 단일 배아 이식 여부, Importance: 1.2057, Unique Values: 3
Feature: 동결 배아 사용 여부, Importance: 1.1672, Unique Values: 3
Accuracy: 0.7121
Precision: 0.4494
Recall: 0.5081
F1 Score: 0.4770
ROC-AUC Score: 0.7416

Confusion Matrix:
[[44671 12366]
 [ 9774 10095]]


[I 2025-02-27 01:56:52,666] Trial 1 finished with value: 0.7415924318310919 and parameters: {'learning_rate': 0.01356869966622113, 'depth': 8, 'n_estimators': 1718, 'class_weights': 1.8, 'l2_leaf_reg': 7, 'bagging_temperature': 1.0834640156391409}. Best is trial 1 with value: 0.7415924318310919.


...


Top 10 Feature Importances:
Feature: 이식된 배아 수, Importance: 31.9877, Unique Values: 4
Feature: 배아 이식 경과일, Importance: 25.7605, Unique Values: 9
Feature: 시술 당시 나이, Importance: 6.6359, Unique Values: 7
Feature: 난자 채취 경과일, Importance: 5.7514, Unique Values: 1
Feature: 난자 출처, Importance: 3.6854, Unique Values: 3
Feature: 배아_저장_비율, Importance: 3.5729, Unique Values: 305
Feature: 총 생성 배아 수, Importance: 3.0624, Unique Values: 44
Feature: 저장된 배아 수, Importance: 1.8667, Unique Values: 41
Feature: 단일 배아 이식 여부, Importance: 1.7796, Unique Values: 3
Feature: 수집된 신선 난자 수, Importance: 1.3028, Unique Values: 52
Accuracy: 0.7035
Precision: 0.4403
Recall: 0.5441
F1 Score: 0.4867
ROC-AUC Score: 0.7414

Confusion Matrix:
[[43297 13740]
 [ 9059 10810]]


[I 2025-02-27 07:34:02,299] Trial 31 finished with value: 0.7413872945920505 and parameters: {'learning_rate': 0.020645039596133515, 'depth': 8, 'n_estimators': 1459, 'class_weights': 1.9, 'l2_leaf_reg': 3, 'bagging_temperature': 1.0658572479964832}. Best is trial 1 with value: 0.7415924318310919.



Top 10 Feature Importances:
Feature: 이식된 배아 수, Importance: 26.5765, Unique Values: 4
Feature: 배아 이식 경과일, Importance: 26.2055, Unique Values: 9
Feature: 시술 당시 나이, Importance: 7.0420, Unique Values: 7
Feature: 난자 채취 경과일, Importance: 4.4585, Unique Values: 1
Feature: 배아_저장_비율, Importance: 4.1486, Unique Values: 305
Feature: 총 생성 배아 수, Importance: 3.5320, Unique Values: 44
Feature: 난자 출처, Importance: 2.9171, Unique Values: 3
Feature: 수집된 신선 난자 수, Importance: 1.8847, Unique Values: 52
Feature: 시술 시기 코드, Importance: 1.7433, Unique Values: 7
Feature: 저장된 배아 수, Importance: 1.6360, Unique Values: 41


[W 2025-02-27 07:34:30,141] Trial 32 failed with parameters: {'learning_rate': 0.02773419675940707, 'depth': 8, 'n_estimators': 1592, 'class_weights': 2.0, 'l2_leaf_reg': 5, 'bagging_temperature': 0.942118089768043} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\HY\python\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HY\AppData\Local\Temp\ipykernel_18304\340033968.py", line 14, in objective
    roc_auc_score = CAT(params, seed)
  File "C:\Users\HY\AppData\Local\Temp\ipykernel_18304\1047224734.py", line 17, in CAT
    model.fit(X_train, y_train)
  File "c:\Users\HY\python\lib\site-packages\catboost\core.py", line 5201, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\HY\python\lib\site-packages\catboost\core.py", line 2396, in _fit
    self.

KeyboardInterrupt: 

In [ ]:
# Accuracy: 0.6934
# Precision: 0.4308
# Recall: 0.5811
# F1 Score: 0.4948
# ROC-AUC Score: 0.7416

# Confusion Matrix:
# [[41779 15258]
#  [ 8323 11546]]
# [I 2025-02-27 02:18:50,049] Trial 3 finished with value: 0.7415804876147438 and parameters: {'learning_rate': 0.015965490084583357, 'depth': 7, 'n_estimators': 1785, 'class_weights': 2.0, 'l2_leaf_reg': 3, 'bagging_temperature': 1.0763333206193686}. Best is trial 1 with value: 0.7415924318310919.

# Top 10 Feature Importances:
# Feature: 이식된 배아 수, Importance: 37.6440, Unique Values: 4
# Feature: 배아 이식 경과일, Importance: 19.5431, Unique Values: 9
# Feature: 시술 당시 나이, Importance: 6.0880, Unique Values: 7
# Feature: 총 생성 배아 수, Importance: 3.8502, Unique Values: 44
# Feature: 난자 채취 경과일, Importance: 3.8048, Unique Values: 1
# Feature: 배아_저장_비율, Importance: 3.6578, Unique Values: 305
# Feature: 난자 출처, Importance: 2.7129, Unique Values: 3
# Feature: 단일 배아 이식 여부, Importance: 1.9142, Unique Values: 3
# Feature: 저장된 배아 수, Importance: 1.8152, Unique Values: 41
# Feature: 수집된 신선 난자 수, Importance: 1.7614, Unique Values: 52

In [64]:
best_params = {'learning_rate': 0.015965490084583357, 'depth': 7, 'n_estimators': 1785, 'class_weights': [1, 2.0], 'l2_leaf_reg': 3, 'bagging_temperature': 1.0763333206193686}

In [65]:
final_model = CatBoostClassifier(**best_params)
final_model.fit(X, y, cat_features=categorical_cols, verbose=0)

In [2]:
pred_proba = final_model.predict_proba(test_data)[:, 1]
pred_proba

NameError: name 'final_model' is not defined

In [67]:
sample_submission = pd.read_csv('dataset/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [68]:
sample_submission

,ID,probability
0,TEST_00000,0.003469
1,TEST_00001,0.009967
2,TEST_00002,0.270283
3,TEST_00003,0.196649
4,TEST_00004,0.678193
...,...,...
90062,TEST_90062,0.003496
90063,TEST_90063,0.496904
90064,TEST_90064,0.589535
90065,TEST_90065,0.358513


In [69]:
sample_submission.to_csv('submit/submit_hy_0227.csv', index=False)